##### System Libraries

In [1]:
import os
import subprocess
import logging
import re
import copy
import time

##### Required Libraries

In [2]:
try:
    import scapy
except ModuleNotFoundError:
    os.system('pip install scapy')
    import scapy
try:
    import tqdm
except ModuleNotFoundError:
    os.system('pip install tqdm')
    import tqdm
try:
    import requests
except ModuleNotFoundError:
    os.system('pip install requests')
    import requests
logging.getLogger("requests").setLevel(logging.INFO)
logging.getLogger("urllib3").setLevel(logging.INFO)

import scapy.plist
from scapy.all import rdpcap
from tqdm import trange,tqdm

### Analyzer

#### Minor Function Breakdowns
---
##### Generate_Logger
Generates a logger with default parameters for logging critical components and debugging
##### Generate Self IPs
Generates a list of IPs assigned to current device according to a protocol list.
##### List Initializer [V2]
Helper function to shorten and simplify initializing and re-initializing lists with values
##### PCAP_Input
Takes a pcap file as input and returns and stores it in memory
##### Generate Unique IPs
Generates a list of unique IPs found in the PCAP data (excluding self IPs)

In [3]:

class Analyzer:
    if True:
        #Initialization of the logger
        logger=logging.getLogger
        stored_logger_level=[logging.DEBUG]
        stored_protocol_list=["IPv4","IPv6","Gateway"]

        stored_pcap_data=[]

        stored_self_ips=[]
        stored_unique_ips=[]
        stored_blocked_ips=[]

        stored_queried_ips=[]
    def __init__(self,path,protocol_list=stored_protocol_list):
        if True:
            self.logger=self.Generate_Logger(filename='AnalysisCore.log')
            logging.getLogger("requests").setLevel(logging.INFO)
            self.Generate_Self_IPs(protocol_list=protocol_list)
            self.PCAP_Input(path)
            self.Generate_Unique_IPs()
    def Generate_Logger(self,filename='default_logger_name.log'): 
        logging.basicConfig(filename=filename,format='%(asctime)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level[0])
        logger=logging.getLogger()
        logger.debug('Logger initialized.')
        return logger
    def List_Initializer(self,input_list,input_param):
        if isinstance(input_param,list) or isinstance(input_param,scapy.plist.PacketList):
            input_list.clear()
            for item in input_param:
                input_list.append(item)
        else:
            input_list.clear()
            input_list.append(input_param)
    def Generate_Self_IPs(self,protocol_list=stored_protocol_list):
        self.logger.info("Generating self IPs with protocols, "+str(protocol_list))
        self_ips=[]
        process=subprocess.Popen(['ipconfig'],stdout=subprocess.PIPE,text=True,shell=True)
        for line in process.stdout:
            for protocol in protocol_list:
                if re.search(protocol,line.strip()):
                    self_ips.append(line.strip()[line.strip().find(':')+2:])
        self.List_Initializer(self.stored_self_ips,self_ips)
        self.logger.debug('Stored self IPs in stored_self_ips '+str(self_ips))
        return self_ips
    def Query_IP(self,ip):
        self.logger.debug('Querying for IP '+ip)
        response=requests.get(f"http://ip-api.com/json/{ip}")
        data=response.json()
        return data
    def PCAP_Input(self,path):
        self.logger.info('Reading file from path, '+path)
        print('Reading PCAP Data.')
        pcap_data=rdpcap(path)
        self.logger.debug('Stored '+path+' data in stored_pcap_data.')
        self.List_Initializer(self.stored_pcap_data,pcap_data)
        return pcap_data
    def Generate_Unique_IPs(self,pcap_data=stored_pcap_data,self_ips=stored_self_ips):
        self.logger.info('Extracting Unique IPs from PCAP Data.')
        unique_ips=[]
        blocked_ips=self_ips.copy()
        self.logger.debug(str(self_ips.copy()))
        packets=pcap_data
        for packet in tqdm(packets,desc='Analyzing packets for unique IPs'):
            flag=0
            try:
                for ip in blocked_ips:
                    if packet.payload.dst==ip:
                        flag=1
                if flag==0:
                    blocked_ips.append(packet.payload.dst)
                    unique_ips.append(packet.payload.dst)
            except AttributeError:
                broadcast_temp='256.256.256.256'
                for ip in blocked_ips:
                    if ip==broadcast_temp:
                        flag=1
                if flag==0:
                    blocked_ips.append(broadcast_temp)
                    unique_ips.append(broadcast_temp)
        self.List_Initializer(self.stored_unique_ips,unique_ips)
        self.logger.debug('Stored unique IPs in stored_unique_ips.')
        return unique_ips
    def Query_Unique_IPs(self,unique_ips=stored_unique_ips,amount=4):
        self.logger.info(f'Querying unique IPs.')
        queried_unique_ips=[]
        for ip in unique_ips[:min(amount,45)]:
            response=self.Query_IP(ip)
            queried_unique_ips.append(response)
            self.logger.debug(f'Stored query in stored_queried_unique_ips.')
            self.List_Initializer(self.stored_queried_ips,queried_unique_ips)
        return queried_unique_ips

In [4]:
A=Analyzer('PCAP Data/720S-Down-Spo-Misc-Idling.pcap')

Reading PCAP Data.


Analyzing packets for unique IPs: 100%|██████████| 10839/10839 [00:00<00:00, 26113.72it/s]


In [5]:
A.Query_Unique_IPs()

[{'status': 'success',
  'country': 'Canada',
  'countryCode': 'CA',
  'region': 'ON',
  'regionName': 'Ontario',
  'city': 'Toronto',
  'zip': 'M5A',
  'lat': 43.6532,
  'lon': -79.3832,
  'timezone': 'America/Toronto',
  'isp': 'Cloudflare, Inc.',
  'org': 'Cloudflare, Inc.',
  'as': 'AS13335 Cloudflare, Inc.',
  'query': '162.159.134.234'},
 {'status': 'success',
  'country': 'United States',
  'countryCode': 'US',
  'region': 'MO',
  'regionName': 'Missouri',
  'city': 'Kansas City',
  'zip': '',
  'lat': 39.0997,
  'lon': -94.5785,
  'timezone': 'America/Chicago',
  'isp': 'Google LLC',
  'org': 'Google Cloud',
  'as': 'AS15169 Google LLC',
  'query': '35.186.224.45'},
 {'status': 'fail', 'message': 'reserved range', 'query': '239.255.255.250'},
 {'status': 'success',
  'country': 'Sweden',
  'countryCode': 'SE',
  'region': 'AB',
  'regionName': 'Stockholm',
  'city': 'Stockholm',
  'zip': '100 05',
  'lat': 59.3293,
  'lon': 18.0686,
  'timezone': 'Europe/Stockholm',
  'isp': 'A

In [6]:
A.stored_queried_ips[0]

{'status': 'success',
 'country': 'Canada',
 'countryCode': 'CA',
 'region': 'ON',
 'regionName': 'Ontario',
 'city': 'Toronto',
 'zip': 'M5A',
 'lat': 43.6532,
 'lon': -79.3832,
 'timezone': 'America/Toronto',
 'isp': 'Cloudflare, Inc.',
 'org': 'Cloudflare, Inc.',
 'as': 'AS13335 Cloudflare, Inc.',
 'query': '162.159.134.234'}

In [13]:
summarized=[]
temp={}
data=A.stored_queried_ips
for query in data:
    try:
        temp={}
        temp['Location']=f"{query['city']}, {query['regionName']}, {query['country']} | Lat: {query['lat']}, Long: {query['lon']}"
        temp['Organization']=f"{query['as']}, {query['org']}, {query['isp']}"
        temp['IP']=f"{query['query']}"
        summarized.append(temp)
    except:
        temp={}
summarized

[{'Location': 'Toronto, Ontario, Canada | Lat: 43.6532, Long: -79.3832',
  'Organization': 'AS13335 Cloudflare, Inc., Cloudflare, Inc., Cloudflare, Inc.',
  'IP': '162.159.134.234'},
 {'Location': 'Kansas City, Missouri, United States | Lat: 39.0997, Long: -94.5785',
  'Organization': 'AS15169 Google LLC, Google Cloud, Google LLC',
  'IP': '35.186.224.45'},
 {'Location': 'Stockholm, Stockholm, Sweden | Lat: 59.3293, Long: 18.0686',
  'Organization': 'AS16509 Amazon.com, Inc., AWS EC2 (eu-north-1), Amazon.com, Inc.',
  'IP': '51.21.215.65'}]

In [20]:
processes=[]
hops=30
timeout=1000
for summary in summarized[:1]:
    print(summary["IP"])
    processes.append(subprocess.Popen(["tracert","-h",str(hops),"-w",str(timeout),summary["IP"]],stdout=subprocess.PIPE,text=True,shell=True))
time.sleep(hops*timeout/1000)
outputs=[]
for process in processes:
    outputs.append(process.stdout)

162.159.134.234


In [24]:
for line in outputs[0]:
    print(line.strip())

In [8]:
# unique_ips=[]
# temp=[]
# ip=A.stored_unique_ips[1]
# response=requests.get(f"http://ip-api.com/json/{ip}")
# data=response.json()
# temp.clear()
# temp.append(ip)
# temp.append(data)
# unique_ips.append(temp)
# unique_ips